### CNN using fits files

In [1]:
# imports
from __future__ import print_function
import os
import keras
from keras import utils as np_utils
import tensorflow
from keras import datasets, layers, models
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
from matplotlib import pyplot as plt

### Preventing Out of memory error (OOM) - Idk why seems like good practice for Mac

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU') # Grabbing all the GPUs avaiable on our machine

# Limit the memory use for all of our GPUs
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

### Find objects in fits images, rotating, etc.
Requieres Clustar

In [ ]:
"""

import joblib
from skimage.transform import resize
from skimage.io import imread
from init_modules import *


# Set the path to the data directory
data_set = {}
im_size = 100
shift_interval = 1

# This function finds the position of the object in the image
def find_object_pos(file):
    cd = ClustarData(path=file, group_factor=0)
    if len(cd.groups) > 0:
        disk = cd.groups[0]
        bounds = disk.image.bounds
        x = (bounds[2] + bounds[3])/2
        y = (bounds[0] + bounds[1])/2
        return (x, y)
    else:
        # img_data = fits.getdata(file)
        
        # return img_data.data.shape[0]/2, img_data.data.shape[1]/2
        print("No object found in {}".format(file))
        return None


# This function crops the image to the size of the object
def init_cropped_images(directory_of_fits_files):
    fits_files = []
    for fits_file in directory_of_fits_files:
        img_data = fits.getdata(fits_file)
        object_pos = find_object_pos(fits_file)

        if object_pos != None:
            # Data shape is (1, 1, x, y) we want it to be (x, y)
            img_data.shape = (img_data.shape[2], img_data.shape[3])

            # Set the size of the crop in pixels
            crop_size = units.Quantity((im_size, im_size), units.pixel)

            img_crop = Cutout2D(img_data, object_pos, crop_size)

            fits_files.append(img_crop)

    return fits_files


# This function rotates the image by a random angle and shifts it by a random amount in a random direction
def rotate_disk(disk_to_rotate, angle):

    # Rotate the disk
    rotated_disk = rotate(disk_to_rotate, angle)
    # Since rotating pads the image, we need to crop it to the original size
    x, y = (len(rotated_disk[0]), len(rotated_disk))

    shift_interval = 0
    si = shift_interval + 1

    rand_x_shift = random.randint(-shift_interval, shift_interval)
    rand_y_shift = random.randint(-shift_interval, shift_interval)

    (x_lower, x_upper) = int((x/2 - im_size/2)) + \
        rand_x_shift, int(x/2 + im_size/2) + rand_x_shift
    (y_lower, y_upper) = int((y/2 - im_size/2)) + \
        rand_y_shift, int(y/2 + im_size/2) + rand_y_shift

    return rotated_disk[(x_lower+si):(x_upper-si), (y_lower+si):(y_upper-si)]


# This function flips the image horizontally, vertically or both
def flip_disk(disk_to_flip):

    flipped_disk = disk_to_flip

    if bool(random.getrandbits(1)):
        flipped_disk = np.fliplr(flipped_disk)

    if bool(random.getrandbits(1)):
        flipped_disk = np.flipud(flipped_disk)

    if bool(random.getrandbits(1)):
        flipped_disk = np.flip(flipped_disk)

    return flipped_disk


# This function augments the image by rotating and flipping it
def augment_disk(disk):
    angle = random.randint(0, 360)
    return rotate_disk(flip_disk(disk), angle)

"""

### Generate dataset from fits files
Requieres Clustar since it runs the above functions

In [3]:
"""

# Generate dataset from the fits files

def generate_dataset(augmentations_per_gaussian, directory_of_fits_files):
    count = 0
    dataset = []
    fits_files = init_cropped_images(directory_of_fits_files)
    for fits_file in fits_files:
        for i in range(0, augmentations_per_gaussian):
            if len(fits_file.data) != 0:
                # zscale = ZScaleInterval(contrast=0.25, nsamples=1)
                
                # ret_data = zscale()
                dataset.append(augment_disk(fits_file.data))
            print(count)
            count += 1
    return dataset

"""

'\n\n# Generate dataset from the fits files\n\ndef generate_dataset(augmentations_per_gaussian, directory_of_fits_files):\n    count = 0\n    dataset = []\n    fits_files = init_cropped_images(directory_of_fits_files)\n    for fits_file in fits_files:\n        for i in range(0, augmentations_per_gaussian):\n            if len(fits_file.data) != 0:\n                # zscale = ZScaleInterval(contrast=0.25, nsamples=1)\n                \n                # ret_data = zscale()\n                dataset.append(augment_disk(fits_file.data))\n            print(count)\n            count += 1\n    return dataset\n\n'

### Configuring/formatting of the dataset 

In [ ]:
"""

# Hyper-parameters data-loading and formatting
nmbr_of_aug = 15
batch_size = 128
num_classes = 2
epochs = 3
img_rows, img_cols = 82, 82  # sqrt of 6724

x_train = np.array(generate_dataset(nmbr_of_aug, glob.glob('data/train_pos/*.fits')) +
                   generate_dataset(nmbr_of_aug, glob.glob('data/train_neg/*.fits')))

batch_size = int(len(x_train)/10)

#print(len(x_train))
#print(x_train[0])
#print(x_train[0].shape)


lbl_train = [0] * len(generate_dataset(nmbr_of_aug, glob.glob('data/train_pos/*.fits'))) + \
            [1] * len(generate_dataset(nmbr_of_aug, glob.glob('data/train_neg/*.fits')))


x_test = np.array(generate_dataset(nmbr_of_aug, glob.glob('data/test_pos/*.fits')) +
                  generate_dataset(nmbr_of_aug, glob.glob('data/test_neg/*.fits')))

lbl_test =  [0] * len(generate_dataset(nmbr_of_aug, glob.glob('data/test_pos/*.fits'))) + \
            [1] * len(generate_dataset(nmbr_of_aug, glob.glob('data/test_neg/*.fits')))

"""

### Converting our numpy arrays describing the fits files to a list of tensor and then making the entire list a tensor itself.

For some reason will some of the augmented images be of a different size than the majority of the files, which we as of now can delete manually as we go. But this needs to be looked into.

In [ ]:
"""
y_train = keras.utils.np_utils.to_categorical(lbl_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(lbl_test, num_classes)


#Convert x_train and x_test to tensors

X_train = []
X_test = []

for arr in x_train:
    X_train.append(tf.convert_to_tensor(arr))

for arr in x_test:
    X_test.append(tf.convert_to_tensor(arr))

X_train = tf.stack(X_train, axis=0, name='stack1')
X_test = tf.stack(X_test, axis=0, name='stack2')

"""

### Build the model

In [ ]:
"""

## Define model ##
model = Sequential()

models_sizes = [0.000001, 0.00001, 0.00005, 0.0001, 0.001]

# print(x_train[500])

epochs = 82

model.add(layers.Conv2D(filters=82, kernel_size=(5, 5),
          activation='relu', input_shape=(98, 98, 1)))
model.add(layers.AveragePooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(filters=128, kernel_size=(2, 2), activation='relu'))
model.add(layers.AveragePooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.summary()

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.SGD(learning_rate=0.1),
              metrics=['accuracy'],)

fit_info = model.fit(X_train, y_train,
                     batch_size=batch_size,
                     epochs=epochs,
                     verbose=1,
                     validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test, verbose=0, return_dict=True)

print(score)

"""